In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime, timedelta
import threading

import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price, convert_lots_to_usd
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp, get_order_from_ticket, get_deal_from_ticket
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import CRYPTOCURRENCIES, CURRENCY_PAIRS, MT5Timeframe
from sesto.telegram import TelegramSender

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
Telegram = TelegramSender()

In [3]:
# PAIRS = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
PAIRS = CRYPTOCURRENCIES + CURRENCY_PAIRS
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
LEVERAGE = 500
DEVIATION = 20
VOLUME = 0.1

TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.06, 'new_sl_pnl_multiplier': 0.025},
]

In [4]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

In [5]:
def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

In [6]:
# dict to store trades, keys are position tickets, values are the entire position object
trades = {}

In [7]:
# Utility Functions
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

# Trading Logic Functions
def check_fractal_signals():
    while True:
        for pair in PAIRS:
            if have_open_positions_in_symbol(pair):
                continue

            df = fetch_data_pos(pair, MAIN_TIMEFRAME, 50)
            df['fractal'] = high_low_finder(df)
            df['symbol'] = pair
            # display(df.tail())

            price = mt5.symbol_info_tick(pair).ask
            new_trade_capital = calculate_trade_capital(pair, VOLUME, LEVERAGE, price)
            trade_size = calculate_position_size(new_trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 

            last_row = df.iloc[-2]
            current_time = datetime.now().replace(microsecond=0)


            if last_row['fractal'] == 'top' or last_row['fractal'] == 'bottom':
                sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='long' if last_row['fractal'] == 'top' else 'short')
                order = send_market_order(symbol=pair, volume=VOLUME, order_type='sell', sl=sl, deviation=DEVIATION)
                if order is not None:
                    trade_info = {
                        'event': 'trade_opened',
                        'entry_condition': f"{last_row['fractal'].upper()} FRACTAL DETECTED",
                        'type': 'long' if last_row['fractal'] == 'top' else 'short',
                        'row_close': f"${last_row['close']:.5f}",
                        'last_tick_price': f"${price:.5f}",
                        'entry_price': f"${order.price:.5f}",
                        'sl': f"${sl:.5f}",
                        'pnl_at_sl': f"${get_pnl_at_price(sl, price, trade_size, LEVERAGE, 'long' if last_row['fractal'] == 'top' else 'short'):.5f}",
                        'symbol': pair,
                        'volume': VOLUME,
                        'time': current_time.isoformat()  # Convert datetime to ISO format string
                    }
                    Telegram.send_json_message(trade_info)
                    display(order)
                else:
                    message = f"{current_time} - {pair} - FAILED TO OPEN POSITION"
                    print(message)
                    Telegram.send_message(message)
                

        # Wait until the next 15-minute interval
        now = current_time
        next_run = (now.replace(second=0, microsecond=0) + pd.Timedelta(minutes=15 - now.minute % 15)).replace(second=0, microsecond=0)
        next_run += pd.Timedelta(seconds=2)  # Add 2 seconds

        sleep_time = (next_run - now).total_seconds()
        print(f"{now} - Waiting for {sleep_time} seconds until next 15-minute check.")
        sleep(sleep_time)

def monitor_open_trades():
    while True:
        current_time = datetime.now().replace(microsecond=0)

        positions = get_positions()
        positions['time'] = pd.to_datetime(positions['time'], unit='s')
        positions['time_update'] = pd.to_datetime(positions['time_update'], unit='s')

        # Create a list to store tickets of closed trades
        closed_trades = []

        for index, position in positions.iterrows():
            # Check if the position ticket exists in trades dict
            if position.ticket not in trades:
                trades[position.ticket] = position
            else:
                # Check if position has changed
                if not trades[position.ticket].equals(position):
                    trades[position.ticket] = position
            
            
            # Calculate the actual capital used for this trade
            trade_capital = calculate_trade_capital(position.symbol, position.volume, LEVERAGE, position.price_open)
            trade_size = calculate_position_size(trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 
            current_pnl_percentage = (position.profit / trade_capital) * 100
            current_sl_pnl = get_pnl_at_price(position.sl, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')

            for trailing_step in TRAILING_STOP_STEPS:
                trigger_pnl_multiplier = trailing_step['trigger_pnl_multiplier']
                new_sl_pnl_multiplier = trailing_step['new_sl_pnl_multiplier']

                trigger_price = get_price_at_pnl(
                    pnl_multiplier=trigger_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )
                new_sl = get_price_at_pnl(
                    pnl_multiplier=new_sl_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )

                trigger_pnl = get_pnl_at_price(
                    trigger_price,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )

                pnl_at_new_sl = get_pnl_at_price(
                    new_sl,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )

                if position.profit >= trigger_pnl:                   
                    if (position.type == 0 and new_sl > position.sl) or (position.type == 1 and new_sl < position.sl):
                        # create a dict with all data regarding old and new sl
                        sl_info = {
                            'event': 'trailing_stop_triggered',
                            'position_data': {
                                'symbol': position.symbol,
                                'trade_open_date': position.time.isoformat(),
                                'type': 'long' if position.type == 0 else 'short',
                                'entry_price': f"${position.price_open:.5f}",
                                'current_price': f"${position.price_current:.5f}",
                                'capital_used': f"${trade_capital:.5f}",
                                'trade_size': f"${trade_size:.5f}",
                                'fee': f"${fee:.5f}",
                            },
                            'trigger_data': {
                                'trigger_price': f"${trigger_price:.5f}",
                                'trigger_pnl': f"${trigger_pnl:.5f}",
                                'trigger_pnl_percentage': f"{(trigger_pnl / trade_capital) * 100:.5f}%",
                            },
                            'current_pnl': {
                                'current_pnl': f"${position.profit:.5f}",
                                'current_pnl_percentage': f"{current_pnl_percentage:.5f}%",
                            },
                            'old_sl': {
                                'old_sl': f"${position.sl:.5f}",
                                'pnl_at_old_sl': f"${current_sl_pnl:.5f}",
                                'old_sl_pnl_percentage': f"{(current_sl_pnl / trade_capital) * 100:.5f}%",
                            },
                            'new_sl': {
                                'new_sl': f"${new_sl:.5f}",
                                'pnl_at_new_sl': f"${pnl_at_new_sl:.5f}",
                                'new_sl_pnl_percentage': f"{(pnl_at_new_sl / trade_capital) * 100:.5f}%",
                            }
                        }

                        modify_request = modify_sl_tp(position.ticket, new_sl, position.tp)
                        if modify_request is not None:
                            Telegram.send_json_message(sl_info)
                        break
               

        # Loop through our trades to see if they are not present in position meaning they have been closed
        for trade_ticket, trade in trades.items():
            if trade_ticket not in positions['ticket'].values:
                message = f'{datetime.now()} - {trade.symbol} - POSITION CLOSED - TICKET: {trade_ticket} - PNL: ${trade.profit:.3f}'
                print(message)
                Telegram.send_message(message)

                sleep(1)

                # Create a dict with all data regarding the closed trade
                closed_order = get_order_from_ticket(trade_ticket)
                if closed_order is not None:
                    closed_order['event'] = 'trade_closed_order'
                    Telegram.send_json_message(closed_order)

                closed_deal = get_deal_from_ticket(trade_ticket, datetime.now() - timedelta(days=1), datetime.now())
                if closed_deal is not None:
                    closed_deal['event'] = 'trade_closed_deal'
                    Telegram.send_json_message(closed_deal)

                closed_trades.append(trade_ticket)

        # Remove closed trades outside the loop
        for ticket in closed_trades:
            del trades[ticket]

        sleep(10)

In [8]:
# Starting Threads
if __name__ == "__main__":
    fractal_thread = threading.Thread(target=check_fractal_signals, daemon=True)
    monitor_thread = threading.Thread(target=monitor_open_trades, daemon=True)

    fractal_thread.start()
    monitor_thread.start()

    try:
        while True:
            sleep(1)
    except KeyboardInterrupt:
        print("Program terminated by user.")

Order successfully placed for BITCOIN


OrderSendResult(retcode=10009, deal=193631941, order=228941380, volume=0.1, price=62285.6, bid=0.0, ask=0.0, comment='Request executed', request_id=813637775, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BITCOIN', volume=0.1, price=62285.6, stoplimit=0.0, sl=62316.7388137216, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

Order successfully placed for DOGECOIN


OrderSendResult(retcode=10009, deal=193631943, order=228941382, volume=0.1, price=0.10925, bid=0.0, ask=0.0, comment='Request executed', request_id=813637776, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='DOGECOIN', volume=0.1, price=0.10925, stoplimit=0.0, sl=0.10992492796832, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

Order failed, retcode: 10016
2024-10-05 01:35:52 - RIPPLE - FAILED TO OPEN POSITION
Order successfully placed for UNISWAP


OrderSendResult(retcode=10009, deal=193631944, order=228941383, volume=0.1, price=6.727, bid=0.0, ask=0.0, comment='Request executed', request_id=813637777, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='UNISWAP', volume=0.1, price=6.727, stoplimit=0.0, sl=6.741368568767999, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

Order successfully placed for CHAINLINK


OrderSendResult(retcode=10009, deal=193631945, order=228941384, volume=0.1, price=11.08, bid=0.0, ask=0.0, comment='Request executed', request_id=813637778, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='CHAINLINK', volume=0.1, price=11.08, stoplimit=0.0, sl=11.105549289599999, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

Order failed, retcode: 10019
2024-10-05 01:35:56 - COSMOS - FAILED TO OPEN POSITION
Order failed, retcode: 10018
2024-10-05 01:35:57 - EURCAD - FAILED TO OPEN POSITION
2024-10-05 01:35:59 - Waiting for 543.0 seconds until next 15-minute check.
SL/TP modified for ticket 228941380 successfully.
Failed to modify SL/TP for ticket 228941380: No changes
SL/TP modified for ticket 228941380 successfully.
Failed to modify SL/TP for ticket 228941380: No changes
SL/TP modified for ticket 228941383 successfully.
Failed to modify SL/TP for ticket 228941380: No changes
2024-10-05 01:37:01.000780 - UNISWAP - POSITION CLOSED - TICKET: 228941383 - PNL: $0.100
No deal history found for ticket 228941383
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Program terminated by user.


Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
Failed to modify SL/TP for ticket 228941380: No changes
2024-10-05 01:39:03.600858 - BITCOIN - POSITION CLOSED - TICKET: 228941380 - PNL: $3.910
No deal history found for ticket 228941380
2024-10-05 01:41:25.919594 - DOGECOIN - POSITION CLOSED - TICKET: 228941382 - PNL: $-6.600
No deal history found for ticket 228941382
Order failed, retcode: 10016
2024-10-05 01:45:02 - RIPPLE - FAILED TO OPEN POSITION
Order failed, retcode: 10018
2024-10-05 01:45:06 - EURCAD - FAILED TO OPEN POSITION
2024-10-05 01:45:07 - Waiting for 895.0 seconds until next 15-minute check.
